In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType
from pyspark.sql import functions as F
from pyspark.sql.functions import date_format
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
from urllib.request import urlretrieve
from zipfile import ZipFile
# Import libraries 

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
import geopandas as gpd
import folium
from chart_studio import plotly as py
import plotly.graph_objs as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import os

/Users/Kasturi/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [1]:

#--------------------------------------------------------------------------------------------
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

22/09/11 20:34:51 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/09/11 20:34:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/11 20:34:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [48]:
exchange_rates = spark.read.csv("../data/exchange_rates.csv")
inflation = spark.read.csv("../data/inflation.csv")
merch_fees = spark.read.csv("../data/merch_fees.csv")
covid = spark.read.csv("../data/covid.csv", header=True)

In [49]:
covid.show(5)

+----------+--------------------+------------+---------+------+-----+---------+---------+----+---+----+--------+
|      date|               state|state_abbrev|confirmed|deaths|tests|positives|recovered|hosp|icu|vent|vaccines|
+----------+--------------------+------------+---------+------+-----+---------+---------+----+---+----+--------+
|2020-01-25|Australian Capita...|         ACT|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|     New South Wales|         NSW|        3|     0|    0|        3|        0|   0|  0|   0|       0|
|2020-01-25|  Northern Territory|          NT|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|          Queensland|         QLD|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|     South Australia|          SA|        0|     0|    0|        0|        0|   0|  0|   0|       0|
+----------+--------------------+------------+---------+------+-----+---------+---------+----+--

In [11]:
data = spark.read.parquet("../data/tables/full_join.parquet")

In [12]:
data.show(5)

+-------------+------------+--------------------+---------+--------------+---------------+--------------------+-----+--------+------+------------------+------------------+--------------------+--------------+-------+-----------+
|merchant_name|merchant_abn|          categories|take_rate|revenue_levels|           name|             address|state|postcode|gender|trans_merchant_abn|      dollar_value|            order_id|order_datetime|user_id|consumer_id|
+-------------+------------+--------------------+---------+--------------+---------------+--------------------+-----+--------+------+------------------+------------------+--------------------+--------------+-------+-----------+
|Felis Limited| 10023283211|furniture, home f...|     0.18|             e|      Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|       10023283211| 212.8573179553598|2430fff1-36c1-483...|    2021-05-02|   3471|        551|
|Felis Limited| 10023283211|furniture, home f...|     0.18|             e|  Adrian Mille

In [13]:
data.createOrReplaceTempView("temp")

a = spark.sql(""" 

SELECT DISTINCT(merchant_name)
FROM temp

""")

a.show()

+--------------------+
|       merchant_name|
+--------------------+
|      Amet Nulla Ltd|
|Volutpat Nulla In...|
|   Pede Praesent Ltd|
|Etiam Bibendum In...|
|Magna Ut Incorpor...|
|Faucibus Leo In C...|
|Varius Et Associates|
|Tellus Imperdiet ...|
|Molestie Tellus F...|
|Class Aptent Taci...|
|Placerat Orci Ass...|
|Vitae Purus Gravi...|
|    Dictum Magna Ltd|
|Sed Leo Cras Inco...|
|    Dui Incorporated|
|        Et Pede Inc.|
| Sagittis Associates|
|Ante Nunc Mauris ...|
|Imperdiet Ullamco...|
|     Ut Molestie LLP|
+--------------------+
only showing top 20 rows



In [56]:
# Code adapted from MAST30034 Tutorial 1
# from the current `tute_1` directory, go back two levels to the `MAST30034` directory
output_relative_dir = '../data/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

# Define the directory names
dirs = ['SA2_boundaries', 'SA2_total_population', 'SA2_income', 'SA2_census']

# now, for each type of data set we will need, we will create the paths
for target_dir in dirs: # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [57]:
SA2_URL_ZIP = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip"

# Define the file names
output_zip = "../data/SA2_boundaries/SA2.zip"

# Download the data
urlretrieve(SA2_URL_ZIP, output_zip)


# Extracting the zip file of the geospatial data

# specifying the zip file name
file_name = "../data/SA2_boundaries/SA2.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # extracting all the files
    zip.extractall(path = "../data/SA2_boundaries/")

In [208]:
income = pd.read_excel("../data/SA2_income/SA2_income.xlsx", sheet_name="Table 1.4")

In [201]:
income

,Australian Bureau of Statistics,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,Personal Income in Australia Table 1. Total In...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Released at 11:30 am (Canberra time) 22/01/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Table 1.4,Total Income by Statistical Area Level 2 (2014...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Earners (persons),NaN,NaN,NaN,NaN,Median age of earners (years),NaN,NaN,...,NaN,NaN,NaN,NaN,Mean ($),NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,801111140,ACT - South West,349,289,347,364,355,40,39,39,...,62592,55384,61096,64227,62505,72858,70503,67445,73435,NaN
2302,801111141,Namadgi,np,np,21,19,18,np,np,34,...,np,49981,58498,40479,np,np,57309,56357,39278,NaN
2303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2304,Totals may not align with the sum of their com...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
income = income.iloc[6:,[0,1,12,13,14,15,16]]

In [211]:
cols_income = ['SA2', "SA2_name", "2014-2015", "2015-2016", "2016-2017", "2017-2018", "2018-2019"]
income.columns = cols_income

In [212]:
income

,SA2,SA2_name,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019
6,Australia,NaN,799748049060,827875051087,856159475083,903888674402,951373381486
7,New South Wales,NaN,256928457330,270324902216,283275046025,300176988652,314465961966
8,101021007,Braidwood,101830606,109961530,115564780,119439615,120763285
9,101021008,Karabar,289762288,301618607,311923113,322715297,338308979
10,101021009,Queanbeyan,367366063,390044004,395374979,414579067,441160946
...,...,...,...,...,...,...,...
2301,801111140,ACT - South West,21814141,21056054,24464466,24549967,26069449
2302,801111141,Namadgi,np,np,1203487,1070776,706999
2303,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2304,Totals may not align with the sum of their com...,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
for index, rows in income.iteritems():
    if index != 'SA2_name':
        for value in rows.values:
            if type(value) == str: 
                income = income[income[index] != value]
        

In [217]:
income.dropna(inplace=True)
income

,SA2,SA2_name,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019
8,101021007,Braidwood,101830606,109961530,115564780,119439615,120763285
9,101021008,Karabar,289762288,301618607,311923113,322715297,338308979
10,101021009,Queanbeyan,367366063,390044004,395374979,414579067,441160946
11,101021010,Queanbeyan - East,186867083,199164715,211520090,232611414,237035656
12,101021011,Queanbeyan Region,717452753,790019714,875609556,964410618,1050285692
...,...,...,...,...,...,...,...
2293,801091109,Phillip,127550251,144908476,152627566,176445396,202204199
2294,801091110,Torrens,101377942,101284791,108571370,114180929,124494941
2296,801101135,Coombs,20041745,70607603,115362521,137389722,179834174
2300,801101139,Wright,85871677,139012922,161448145,171246822,182421537


In [216]:
income.to_csv("../data/curated/SA2_income.csv")